In [7]:
import pandas as pd
path = "../data/raw/Online_Retail.xlsx"
df = pd.read_excel(path, sheet_name=0)
print(df.shape)
print(df.columns.tolist())
df.head()

(541909, 8)
['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [8]:
# 1. Remove rows with missing CustomerID
df = df[df['CustomerID'].notnull()]

# 2. Remove negative or zero quantities/prices
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# 3. Remove cancelled invoices (start with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# 4. Create a 'TotalPrice' column (Quantity × UnitPrice)
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# 5. Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


In [9]:
print(df.shape)         
print(df.isnull().sum())
df.head()


(397884, 9)
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
TotalPrice     0
dtype: int64


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [10]:
import datetime as dt

# 1. Set a reference date (last date in dataset + 1 day)
reference_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

# 2. Group by CustomerID and calculate RFM metrics
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  
    'InvoiceNo': 'nunique',                                   
    'TotalPrice': 'sum'                                    
})

# 3. Rename columns
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPrice': 'Monetary'
}, inplace=True)

# 4. Preview RFM table
print(rfm.shape)
rfm.head()


(4338, 3)


,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,1,77183.60
12347.0,2,7,4310.00
12348.0,75,4,1797.24
12349.0,19,1,1757.55
12350.0,310,1,334.40


In [11]:
# 1. Score each metric using quartiles
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4,3,2,1])  
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1,2,3,4])  
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1,2,3,4])  

# 2. Combine into one string RFM Score
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# 3. Define segments (simplified version)
def segment_me(df):
    if df['RFM_Score'] in ['444','434','344','443']:
        return 'Champions'
    elif df['R_Score'] in ['3','4'] and df['F_Score'] in ['3','4']:
        return 'Loyal Customers'
    elif df['R_Score'] in ['4'] and df['F_Score'] in ['1','2']:
        return 'Potential Loyalists'
    elif df['R_Score'] in ['2','3'] and df['F_Score'] in ['1','2']:
        return 'At Risk'
    else:
        return 'Others'

rfm['Segment'] = rfm.apply(segment_me, axis=1)

# 4. Preview result
rfm[['Recency','Frequency','Monetary','RFM_Score','Segment']].head(10)


,Recency,Frequency,Monetary,RFM_Score,Segment
CustomerID,,,,,
12346.0,326,1,77183.60,114,Others
12347.0,2,7,4310.00,444,Champions
12348.0,75,4,1797.24,234,Others
12349.0,19,1,1757.55,314,Others
12350.0,310,1,334.40,112,Others
12352.0,36,8,2506.04,344,Champions
12353.0,204,1,89.00,111,Others
12354.0,232,1,1079.40,113,Others
12355.0,214,1,459.40,112,Others


In [12]:
# export the RFM table
rfm.to_csv("../data/processed/rfm_table.csv", index=True)
